# USE "TPU v5e-1" to handle with the full dataset


In [ ]:
# Update Kaggle lib and put credentials


In [ ]:
# %pip install --quiet kaggle --upgrade


To use the Kaggle API, sign up for a Kaggle account at https://www.kaggle.com. <br> Then go to Settings (https://www.kaggle.com/settings) and select 'Create API Token'.


In [ ]:
# Download data


In [ ]:
import os
os.environ['KAGGLE_USERNAME'] = ''
os.environ['KAGGLE_API_TOKEN'] = ''


In [3]:
!kaggle competitions download -c mipt-gnn-2025


zsh:1: command not found: kaggle


!unzip mipt-gnn-2025.zip


# Install PyG


In [8]:
import os

def install_libs(torch_version_to_use):
  os.environ['TORCH'] = torch_version_to_use

  !pip3 install torch==${TORCH} --upgrade
  import torch
  if torch.cuda.is_available():
      CUDA = f'{torch.version.cuda}'
      print(f"CUDA version used by PyTorch: {CUDA}")
      os.environ['CUDA'] = f'cu{CUDA.replace(".","")}'

  !pip uninstall -y torch_geometric pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv
  !pip install torch_geometric --upgrade
  if torch.cuda.is_available():
    !pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-${TORCH}+${CUDA}.html
  else:
    !pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-${TORCH}+cpu.html

In [9]:
install_libs(torch_version_to_use = "2.8.0")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 MB 8.9 MB/s  0:00:08m0:00:0100:01
  Attempting uninstall: torch
    Found existing installation: torch 2.9.1
    Uninstalling torch-2.9.1:
      Successfully uninstalled torch-2.9.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [torch_geometric] [torch_geometric]
Looking in links: https://data.pyg.org/whl/torch-2.8.0+cpu.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 4.1 MB/s  0:00:00 eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 814.5/814.5 kB 3.2 MB/s  0:00:0036m-:--:--
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.3/599.3 kB 2.1 MB/s  0:00:0036m-:--:--
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 622.2/622.2 kB 18.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [torch_cluster]


In [1]:
import torch
import torch_geometric
print(torch.cuda.is_available())

import numpy as np
import pandas as pd
import sys
import os
import pandas as pd
import numpy as np
import tqdm
import time
import matplotlib.pyplot as plt
%matplotlib inline
import sklearn
import warnings
warnings.filterwarnings('ignore')



/Users/anymax/miniconda3/envs/mobod-py310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


False


# Read raw


In [ ]:
# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))


In [2]:
dirname = './'


In [3]:
import fastparquet
import pyarrow

df_edges = pd.read_parquet(os.path.join(dirname,'df_edges.parquet'), engine='fastparquet')
df_nodes = pd.read_parquet(os.path.join(dirname,'df_nodes.parquet'), engine='fastparquet')


In [4]:
df_nodes.head(2)


,node_feature_1,node_feature_2,node_feature_3,node_feature_4,node_feature_5,node_feature_6,node_feature_7,node_feature_8,node_feature_9,node_feature_10,...,node_feature_237,node_feature_238,node_feature_239,node_feature_240,node_feature_241,node_feature_242,node_feature_243,node_feature_244,node_feature_245,index
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,0.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,NaN,1.0,...,1.753344,2.027835,0.0,0.0,0.0,0.0,0.0,0.429199,0.0,1


In [5]:
node_group_col = 'node_feature_1'
del df_nodes['index']


In [6]:
print(df_nodes['node_feature_1'].value_counts().sort_index())


node_feature_1
0.0    1410424
1.0    1934612
Name: count, dtype: int64


In [ ]:
a = df_nodes.isnull()#.sum(axis=0)

b = a[df_nodes[node_group_col]==0].sum(axis=0)
features_group0 = b[b==0].index.tolist()
features_group0.remove(node_group_col)

b = a[df_nodes[node_group_col]==1].sum(axis=0)
features_group1 = b[b==0].index.tolist()
features_group1.remove(node_group_col)

for col in df_nodes.columns:
    if col != node_group_col:
        for group_val in [0, 1]:
            mask = df_nodes[node_group_col] == group_val
            if mask.sum() > 0:
                median_val = df_nodes.loc[mask, col].median()
                if pd.notna(median_val):
                    df_nodes.loc[mask & df_nodes[col].isna(), col] = median_val
                else:
                    df_nodes.loc[mask & df_nodes[col].isna(), col] = 0.0

del a


In [8]:
print("Число фичей для группы вершин 0:",len(features_group0))
print("Число фичей для группы вершин 1:",len(features_group1))


Число фичей для группы вершин 0: 134
Число фичей для группы вершин 1: 162


In [9]:
# Найдем индексы этих фичей
cols = df_nodes.columns.tolist()
features_idx_group0 = [cols.index(c) for c in features_group0]
features_idx_group1 = [cols.index(c) for c in features_group1]
df_edges.head(2)


,edge_feature_1,edge_feature_2,edge_feature_3,edge_feature_5,edge_feature_7,edge_feature_9,edge_feature_11,edge_feature_12,edge_feature_13,edge_feature_14,...,edge_feature_41,edge_feature_42,edge_feature_43,edge_feature_44,edge_feature_45,edge_feature_46,edge_feature_47,edge_feature_48,index1,index2
0,1.0,1.0,-1.489233,-2.149860,0.874358,-2.07540,-0.81038,1.519856,0.744896,-0.484251,...,0.133585,0.000000,0.000000,0.000000,0.153324,0.000000,0.0,0.0,14995,0
1,1.0,1.0,0.756959,0.831242,-0.422108,0.88719,-0.81038,0.233813,0.744896,0.829714,...,0.506228,0.299537,0.928534,0.909701,0.969412,0.772556,0.0,0.0,14996,0


In [ ]:
x = torch.FloatTensor(df_nodes.values)
edge_index = torch.LongTensor(df_edges[['index1','index2']].values).T

del df_edges['index1']
del df_edges['index2']

edge_attr = torch.FloatTensor(df_edges.values)

from torch_geometric.data import Data
del df_edges


In [11]:
import gc
gc.collect()


0

### Таргет


In [12]:
df_train = pd.read_csv(os.path.join(dirname,'train.csv'))
df_test = pd.read_csv(os.path.join(dirname,'test.csv'))
# 4 бинарных таргета
# доли 1 в каждом тарегте, число 1, размер выборки по таргету
df_train.groupby('label_type').y.agg(['mean','sum','count'])


,mean,sum,count
label_type,,,
label_3,0.247525,75,303
label_4,0.171748,338,1968
label_5,0.208788,708,3391
label_6,0.088658,401,4523


In [13]:
# размер выборки по таргету для test
df_test.label_type.value_counts()


label_type
label_6    2229
label_5    1671
label_4     969
label_3     149
Name: count, dtype: int64

In [14]:
df_y = df_train.pivot(columns='label_type',index='index',values='y')
df_y.head()


label_type,label_3,label_4,label_5,label_6
index,,,,
2,NaN,NaN,0.0,NaN
3,NaN,NaN,0.0,NaN
4,NaN,0.0,NaN,NaN
5,NaN,0.0,NaN,NaN
9,NaN,NaN,NaN,0.0


In [15]:
df_y_ = pd.DataFrame(np.arange(len(x)) , columns=['index'])
df_y = df_y_.merge(df_y.reset_index(), on='index',how='left').set_index('index')
df_y.head()


,label_3,label_4,label_5,label_6
index,,,,
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,NaN,0.0,NaN
3,NaN,NaN,0.0,NaN
4,NaN,0.0,NaN,NaN


In [16]:
df_y['has_target'] = df_y.notnull().sum(axis=1) > 0
df_y = df_y.fillna(-100)
df_y.head(1)


,label_3,label_4,label_5,label_6,has_target
index,,,,,
0,-100.0,-100.0,-100.0,-100.0,False


In [17]:
node_type = torch.ShortTensor(df_nodes[node_group_col].values)


### Data

In [18]:
# 4 таргета
label_3 = torch.LongTensor(df_y.label_3.values)
label_4 = torch.LongTensor(df_y.label_4.values)
label_5 = torch.LongTensor(df_y.label_5.values)
label_6 = torch.LongTensor(df_y.label_6.values)

#
has_target = torch.BoolTensor(df_y.has_target.values)


In [19]:
# cобираем ключевой объект data
data = Data(x=x, # node features
            edge_index=edge_index, # edges
            edge_attr=edge_attr, # edge attributes
            label_3=label_3,
            label_4=label_4,
            label_5=label_5,
            label_6=label_6,
            has_target=has_target,# whether node has target
            node_type=node_type, # 0 or 1st type of node
            index = torch.arange(len(x))# unique identifier of node
           )


In [20]:
print('===================================')

print(f'Number of nodes: {data.x.shape[0]}')
print(f'Number of edges: {data.edge_index.shape[1]}')
print(f'Number of features: {data.x.shape[1]}')

if data.edge_attr is not None:
    print(f'Number of edge attributes: {data.edge_attr.shape[1]}')


print(f'Number of nodes with target: {int(data.has_target.sum())}')
print('===================================')


Number of nodes: 3345036
Number of edges: 3907086
Number of features: 245
Number of edge attributes: 44
Number of nodes with target: 10046


### Локальная валидация


In [21]:
from sklearn.model_selection import train_test_split
stratify = df_y[df_y['has_target']].drop('has_target',axis=1).max(axis=1)
train_index, valid_index = train_test_split(torch.nonzero(data.has_target)[:,0],
                                            test_size=0.2,
                                            shuffle=True,
                                            stratify=stratify,
                                            random_state=402)
test_index = df_test['index'].unique()
len(train_index), len(valid_index), len(test_index)


(8036, 2010, 4949)

In [ ]:
from torch_geometric.loader import NeighborLoader
batch_size = 200

loader_train = NeighborLoader(
    data,
    num_neighbors=[-1, -1, -1],  # 3 слоя
    batch_size=batch_size,
    shuffle=False,
    input_nodes=train_index,
)

loader_valid = NeighborLoader(
    data,
    num_neighbors=[-1, -1, -1],
    batch_size=batch_size,
    shuffle=False,
    input_nodes=valid_index,
)


loader_test = NeighborLoader(
    data,
    num_neighbors=[-1, -1, -1],
    batch_size=batch_size,
    shuffle=False,
    input_nodes=test_index,
)

# предрасчитываем разбиение графа на батчи
loader_train = [c for c in tqdm.tqdm(loader_train)]
loader_valid = [c for c in tqdm.tqdm(loader_valid)]
loader_test = [c for c in tqdm.tqdm(loader_test)]


100%|██████████| 25/25 [00:00<00:00, 105.12it/s]


### Модель


In [35]:
from torch.nn import Linear, ReLU, Sequential, Identity, Dropout, PReLU, Tanh, ELU, LeakyReLU, Sigmoid, GELU, LayerNorm
from torch_geometric.nn import GraphConv, SAGEConv, GATConv, TransformerConv
import torch


In [ ]:
class GeneralGNN(torch.nn.Module):
    def __init__(self,
                 Act = GELU,
                 ConvClass = GraphConv,
                 hidden_size1 = 256,
                 conv1_aggrs = {'mean':128, 'max':64, 'add':32},
                 conv2_aggrs = {'mean': 64, 'max':16, 'add':8},
                 dropout_rate = 0.5,
                 target_names = ['label_3','label_4','label_5','label_6'],
                 features_idx_group0 = features_idx_group0,
                 features_idx_group1 = features_idx_group1,
                 use_gat = True,
                 edge_dim = None,  # Размерность edge features
                ):
        super(GeneralGNN, self).__init__()
        torch.manual_seed(420)

        self.hidden_size1 = hidden_size1
        self.target_names = target_names
        self.features_idx_group0 = features_idx_group0
        self.features_idx_group1 = features_idx_group1
        self.use_gat = use_gat
        self.edge_dim = edge_dim

        # preprocces node featrues, reduce to a common dimension
        self.lin_preproc = torch.nn.ModuleList(
            modules = [
                Linear(len(features_idx_group0), hidden_size1),
                Linear(len(features_idx_group1), hidden_size1)
            ]
        )

        self.bn_preproc = LayerNorm(hidden_size1)
        self.activation = Act()
        self.dropout = Dropout(dropout_rate)


        # 1-level convolution with different aggregate functions
        input_size = hidden_size1

        if use_gat and edge_dim is not None:
            num_heads = 2
            self.conv1_gat = GATConv(input_size, conv1_aggrs['mean'], heads=num_heads, 
                                    dropout=dropout_rate, edge_dim=edge_dim)
            gat_output_size = conv1_aggrs['mean'] * num_heads
            self.conv1_proj = Linear(gat_output_size, sum(conv1_aggrs.values()))
            self.conv1 = torch.nn.ModuleDict()
            for aggr, output_size in conv1_aggrs.items():
                if aggr != 'mean':
                    self.conv1[aggr] = ConvClass(input_size, output_size, aggr)
        else:
            self.conv1 = torch.nn.ModuleDict()
            for aggr, output_size in conv1_aggrs.items():
                self.conv1[aggr] = ConvClass(input_size, output_size, aggr)
            self.conv1_gat = None
            self.conv1_proj = None

        if use_gat and edge_dim is not None:
            conv1_output_size = sum(conv1_aggrs.values())
            for aggr, output_size in conv1_aggrs.items():
                if aggr != 'mean':
                    conv1_output_size += output_size
        else:
            conv1_output_size = sum(conv1_aggrs.values())

        self.bn1 = LayerNorm(conv1_output_size)
        self.conv1_activation = Act()
        self.conv1_dropout = Dropout(dropout_rate)

        input_size = conv1_output_size

        conv2_output_size = 48
        conv2_heads = 2
        self.conv2 = TransformerConv(
            in_channels=input_size,
            out_channels=conv2_output_size,
            heads=conv2_heads,
            dropout=dropout_rate,
            edge_dim=edge_dim if use_gat and edge_dim is not None else None
        )

        conv2_final_size = conv2_output_size * conv2_heads

        self.bn2 = LayerNorm(conv2_final_size)
        self.conv2_activation = Act()
        self.conv2_dropout = Dropout(dropout_rate * 0.5)
        
        # MAKE PREDICTION FOR EACH TASK
        input_size = conv2_final_size
        self.lin_out = torch.nn.ModuleDict()
        for name in self.target_names:
            self.lin_out[name] = Linear(input_size, 2)


    def forward(self, data):
        x = data.x
        edge_index = data.edge_index
        node_type = data.node_type
        edge_attr = data.edge_attr if self.edge_dim is not None else None

        x_out = torch.zeros(len(x), self.hidden_size1, device=x.device)

        x_out[node_type==0] = self.lin_preproc[0](x[node_type==0][:,self.features_idx_group0])
        x_out[node_type==1] = self.lin_preproc[1](x[node_type==1][:,self.features_idx_group1])

        x = self.bn_preproc(x_out)
        x = self.dropout(self.activation(x))

        x_out = []

        if self.use_gat and self.conv1_gat is not None and edge_attr is not None:
            x_gat = self.conv1_gat(x, edge_index, edge_attr=edge_attr)
            x_gat = self.conv1_proj(x_gat)
            x_out.append(x_gat)
            for conv in self.conv1.values():
                x_out.append(conv(x, edge_index))
        else:
            for conv in self.conv1.values():
                x_out.append(conv(x, edge_index))
        
        x = torch.cat(x_out, dim=1)
        x = self.bn1(x)
        x = self.conv1_dropout(self.conv1_activation(x))

        x = self.conv2(x, edge_index, edge_attr=edge_attr if self.edge_dim is not None else None)
        x = self.bn2(x)
        x = self.conv2_dropout(self.conv2_activation(x))

        outs = {name: self.lin_out[name](x) for name in self.target_names}

        return outs


In [ ]:
model = GeneralGNN(use_gat=True, edge_dim=edge_attr.shape[1])
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)


GeneralGNN(
  (lin_preproc): ModuleList(
    (0): Linear(in_features=134, out_features=256, bias=True)
    (1): Linear(in_features=162, out_features=256, bias=True)
  )
  (bn_preproc): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
  (activation): GELU(approximate='none')
  (dropout): Dropout(p=0.5, inplace=False)
  (conv1_gat): GATConv(256, 128, heads=2)
  (conv1_proj): Linear(in_features=256, out_features=224, bias=True)
  (conv1): ModuleDict(
    (max): GraphConv(256, 64)
    (add): GraphConv(256, 32)
  )
  (bn1): LayerNorm((320,), eps=1e-05, elementwise_affine=True)
  (conv1_activation): GELU(approximate='none')
  (conv1_dropout): Dropout(p=0.5, inplace=False)
  (conv2): TransformerConv(320, 48, heads=2)
  (bn2): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
  (conv2_activation): GELU(approximate='none')
  (conv2_dropout): Dropout(p=0.25, inplace=False)
  (lin_out): ModuleDict(
    (label_3): Linear(in_features=96, out_features=2, bias=True)
    (label_4): Linear(in_f

In [47]:
def train_epoch(model, loader, device, optimizer, scheduler, tqdm_disable=True):
    """
    учим модель одну эпоху
    """
    start_time = time.time()

    model.train()
    total_loss = 0

    for sampled_data in tqdm.tqdm(loader, leave=False, disable=tqdm_disable):
        sampled_data = sampled_data.clone().to(device)
        sampled_data.has_target[sampled_data.batch_size:] = False

        optimizer.zero_grad()
        outs = model(sampled_data)
        loss = get_train_loss(sampled_data, outs)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1)
        optimizer.step()

        total_loss += loss.detach().item() / len(loader)

    scheduler.step()

    calc_time = time.time() - start_time
    return total_loss, calc_time


In [48]:
# считаем лосс как сумму лоссов по каждому таргету
def get_train_loss(sampled_data, outs):
    ys = {name: getattr(sampled_data,name)[:sampled_data.batch_size] for name in target_names}
    outs = {name: outs[name][:sampled_data.batch_size] for name in target_names}
    loss = 0
    for name in target_names:
        y = ys[name]
        out = outs[name]
        l = criterion(out,y)
        if not torch.isnan(l):
            loss +=l
    return loss/batch_size


In [49]:
def eval_model(model, loader, device, tqdm_disable=True):
    """
    get prediction


    """
    start_time = time.time()
    softmax = torch.nn.Softmax(dim=1)

    dfs = []

    model.eval()
    with torch.no_grad():

        for sampled_data in tqdm.tqdm(loader, leave=False, disable=tqdm_disable):
            sampled_data = sampled_data.clone().to(device)
            outs = model(sampled_data)
            outs = {name: softmax(
                                outs[name][:sampled_data.batch_size].detach()
                            )[:,1].cpu().numpy() for name in target_names
                   }
            outs['index'] = sampled_data.index[:sampled_data.batch_size].cpu().numpy()
            dfs.append(pd.DataFrame(outs))

    result = pd.concat(dfs,axis=0)

    df_score = pd.melt(result, id_vars=['index'], value_vars=target_names).rename(
    columns={'variable':'label_type','value':'score'})
    return df_score


In [50]:
from sklearn.metrics import roc_auc_score
def cacl_auc(df_score, df_truth):
    df = df_truth.merge(df_score, on=['index','label_type'])
    return roc_auc_score(df.y, df.score) #, df.shape[0]


In [51]:
target_names = ['label_3','label_4','label_5','label_6']


In [ ]:
criterion = torch.nn.CrossEntropyLoss(reduction='sum', ignore_index=-100, label_smoothing = 0.1)  # было 0.05

optimizer = torch.optim.AdamW(model.parameters(), lr=0.0005, weight_decay=7e-4, betas=(0.9, 0.999))  # было lr=0.0005, weight_decay=5e-4

from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
scheduler = CosineAnnealingWarmRestarts(
      optimizer = optimizer,
      T_0 = 8,  # было 10
      T_mult = 2,
      eta_min = 1e-6)  # было 1e-5


# from torch.optim.lr_scheduler import LambdaLR
# warmup_epochs = 5
# def lr_lambda(epoch):
#     if epoch < warmup_epochs:
#         return (epoch + 1) / warmup_epochs
#     return 1.0
# scheduler = LambdaLR(optimizer, lr_lambda)

### Train

In [ ]:
best_valid_auc = 0
patience = 7
patience_counter = 0
max_epochs = 50

# 1.5 min per epoch
for epoch in range(max_epochs):
    total_loss, calc_time = train_epoch(model, loader_train, device, optimizer, scheduler, tqdm_disable=False)

    df_score = eval_model(model, loader_train, device, tqdm_disable=True)
    train_auc = cacl_auc(df_score, df_train)

    df_score = eval_model(model, loader_valid, device, tqdm_disable=True)
    valid_auc = cacl_auc(df_score, df_train)

    print(f'Epoch:{epoch:03d} TrainLoss : {total_loss:.3f}, Train AUC : {train_auc:.3f}, VALID AUC : {valid_auc:.3f}',
          file=sys.stderr)

    if valid_auc > best_valid_auc:
        best_valid_auc = valid_auc
        patience_counter = 0
        torch.save(model.state_dict(), 'best_model.pt')
        print(f'  -> New best model! Valid AUC: {valid_auc:.3f}', file=sys.stderr)
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print(f'Early stopping at epoch {epoch}', file=sys.stderr)
            break

if os.path.exists('best_model.pt'):
    model.load_state_dict(torch.load('best_model.pt'))
    print(f'Loaded best model with Valid AUC: {best_valid_auc:.3f}', file=sys.stderr)


Epoch:000 TrainLoss : 0.506, Train AUC : 0.660, VALID AUC : 0.683
  -> New best model! Valid AUC: 0.683
Epoch:001 TrainLoss : 0.466, Train AUC : 0.708, VALID AUC : 0.730
  -> New best model! Valid AUC: 0.730
Epoch:002 TrainLoss : 0.460, Train AUC : 0.725, VALID AUC : 0.744
  -> New best model! Valid AUC: 0.744
Epoch:003 TrainLoss : 0.454, Train AUC : 0.739, VALID AUC : 0.752
  -> New best model! Valid AUC: 0.752
Epoch:004 TrainLoss : 0.451, Train AUC : 0.753, VALID AUC : 0.758
  -> New best model! Valid AUC: 0.758
Epoch:005 TrainLoss : 0.450, Train AUC : 0.762, VALID AUC : 0.759
  -> New best model! Valid AUC: 0.759
Epoch:006 TrainLoss : 0.445, Train AUC : 0.765, VALID AUC : 0.761
  -> New best model! Valid AUC: 0.761
Epoch:007 TrainLoss : 0.444, Train AUC : 0.766, VALID AUC : 0.761
Epoch:008 TrainLoss : 0.445, Train AUC : 0.774, VALID AUC : 0.764
  -> New best model! Valid AUC: 0.764
Epoch:009 TrainLoss : 0.443, Train AUC : 0.789, VALID AUC : 0.759
Epoch:010 TrainLoss : 0.440, Train A

In [ ]:
df_score = eval_model(model, loader_test, device, tqdm_disable=True)
df_submission = df_score.merge(df_test, on=['index','label_type'])
df_submission[['task_id','score']].to_csv('submission_5.csv',index=False, header=True)


# Submit


In [39]:
!kaggle competitions submit mipt-gnn-2025 -f submission_3.csv -m "improved baseline with GAT, TransformerConv"


zsh:1: unmatched "
